# Logistic Regression is a 1 Layer Neural Network

## Summary: The purpose of this tutorial is to demystify an increasingly popular method in machine learning: neural networks. Here, the reader will see that logistic regression can be framed as a specific architecture of a neural network. 

## The tutorial is presented in R in an effort to appeal to data professionals who may not consider themselves machine learning practitioners. The only prerequisites are experience with logistic regression and familiarity with calculus concepts.

In [1]:
# Make sure the required packages are installed if they aren't already.

for (package in c("keras", "mlbench")) {
    if (!(package %in% installed.packages()[, "Package"])) {
        install.packages(package)
    }
}

### [Keras](https://keras.io/) is a machine learning library that is a front-end to other deep learning frameworks such as TensorFlow and Theano. Here we import the R interface to Keras.

In [2]:
library(keras)
library(mlbench)

### Read a sample dataset in. Here we're using the [Pima Indians Diabetes Data Set ](https://archive.ics.uci.edu/ml/datasets/pima+indians+diabetes).

In [3]:
data(PimaIndiansDiabetes)

In [4]:
head(PimaIndiansDiabetes)

pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
6,148,72,35,0,33.6,0.627,50,pos
1,85,66,29,0,26.6,0.351,31,neg
8,183,64,0,0,23.3,0.672,32,pos
1,89,66,23,94,28.1,0.167,21,neg
0,137,40,35,168,43.1,2.288,33,pos
5,116,74,0,0,25.6,0.201,30,neg


### Replace variable 'diabetes' with binary {0, 1}.

In [5]:
PimaIndiansDiabetes$diabetes <- ifelse(PimaIndiansDiabetes$diabetes=="pos", 1, 0)

In [6]:
data <- PimaIndiansDiabetes

In [7]:
X <- as.matrix(data[, seq(1,8)])
Y <- as.matrix(data[, 9])


# Below is optional for the given exercise.
sample_size <- floor(0.75 * nrow(data))
train_ind <- sample(seq_len(nrow(data)), size = sample_size)
x_train <- X[train_ind, ]
y_train <- Y[train_ind, ]
x_test <- X[-train_ind, ]
y_test <- Y[-train_ind, ]

In [8]:
# dim(data) = n x k
# dim(weights) = k x 1
# dim(output) = n x 1

# dim(X)[2] = k
inputs <- layer_input(shape = c(dim(X)[2]))
 
# Model will be only 1 layer (besides input) -- the weight layer and the output (argument in layer_dense)
outputs <- inputs %>%
    # activation is sigmoid because we're dealing with logistic regression. It's the same as 
    # logit function -> 1 / (1 + exp(-X %*% W)). We're leaving out intercept (bias).
    layer_dense(units = 1, activation = 'sigmoid', use_bias = FALSE)

### Inputs are of shape (? , 8) -- the model is agnostic about the number of samples but needs to know what shape to make the first layer. 
### layer_input (? x k) %*% layer_dense (k x 1) %>% sigmoid_function -> Output (? x 1)

In [9]:
nn_model <- keras_model(inputs = inputs, outputs = outputs)

### We define our model using binary_crossentropy as the loss function to minimize.

In [10]:
nn_model %>% compile(
  loss = 'binary_crossentropy',
  optimizer = optimizer_rmsprop(),
  metrics = c('binary_crossentropy')
)

### What is binary crossentropy? It's the negative of the log likelihood of a Bernoulli distribution, which is exactly what glm in R is maximizing for logistic regression.


$$binary \ crossentropy = \large - \Sigma_{i=1}^{n} \big( y_i \ (log(\hat{y_i})) + (1 - y_i) \ log(1 - \hat{y_i}) \big)$$

$$y_i = \text{observation} \ i$$
$$\hat{y_i} = \text{predicted probability for observation} \ i$$


$$Bernoulli \ likelihood \ function = \large \Pi_{i=1}^{n} \big( p^{x_i} \ (1 - p)^{(1 - x_i)}    \big) $$

Taking the log of the above we get:

$$ log(Bernoulli \ likelihood \ function)  = \large \Sigma_{i=1}^{n} \big( x_i \ log(p) + ( 1 - x_i)log(1-p) \big)$$

In [11]:
history <- nn_model %>% fit(
  X, Y, 
  epochs = 1000, batch_size = 32
)

### Get the weights from the model. In our model there is only the input layer so we only see an array of dimension (k x 1), just as defined above in layer_dense.

In [12]:
get_weights(nn_model)

0.130209595
0.013974152
-0.029735258
0.001157426
0.001042823
-0.003845388
0.324986279
-0.014955019


### Fit a logistic model to the data, leaving out the intercept as with the neural network. Just like with other machine learning methods, logistic regression in R is doing some optimization under the hood. Specifically, R is using [Iteratively reweighted least squares](https://en.wikipedia.org/wiki/Iteratively_reweighted_least_squares).

In [13]:
logistic_model <- glm(Y ~ X - 1, family = binomial(link='logit'), control = list(maxit = 25, epsilon=1e-10))

In [14]:
logistic_model$coefficients

Xpregnant      Xglucose     Xpressure      Xtriceps      Xinsulin 
 0.1284180545  0.0129358347 -0.0303255466  0.0001956745  0.0007389038 
        Xmass     Xpedigree          Xage 
-0.0048136216  0.3202837749 -0.0156346467

### The coefficients (weights) from the neural network and logistic regression are very nearly identical. Any differences, might be attributable to the difference in number of training iterations or to the different optimization methods used in both models: [RMSprop](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) in the neural network and [Fisher's scoring](https://en.wikipedia.org/wiki/Scoring_algorithm) for glm in R.

In [15]:
get_weights(nn_model)[[1]] - logistic_model$coefficients

0.0017915405
0.0010383169
0.0005902887
0.0009617514
0.0003039189
0.0009682335
0.0047025041
0.0006796281


### You're still skeptical. Let's implement our neural network model from above from scratch, with one exception: instead of minimizing binary crossentropy we're going to maximize log-likelihood.

Recall that 

$$binary \ crossentropy = \large - \Sigma_{i=1}^{n} \big( y_i \ (log(\hat{y_i})) + (1 - y_i) \ log(1 - \hat{y_i}) \big)$$

and since $\hat{y_i} = \frac{1}{1 + \exp^{- X \theta}}$, we obtain

$$\frac{\partial \ binary \ crossentropy}{\partial \ \theta} = \large \Sigma_{i=1}^{n} \big( \hat{y_i} - y_i \big) x_i$$

The derivative of the log-likelihood with respect to $\theta$ is the negative of the above.

In [16]:
n <- dim(X)[1]
k <- dim(X)[2]


sigmoid_fn <- function(z) {
    return (1 / (1 + exp(-z)))
}

theta <- rep(0, k)


# binary_crossentropy <- - sum(Y * log(y_hat) + (1 - Y) * log(1 - y_hat))
# log_likelihood <- sum(Y * log(y_hat) + (1 - Y) * log(1 - y_hat))

alpha <- 0.005
beta <- 0.9

v_dtheta <- 0

for (epoch in 1:1000) {
    
    z <- X %*% theta
    y_hat <- sigmoid_fn(z)
    
    # In a normal neural network we would use the derivative of binary crossentropy, but we're
    # using the derivative of the log-likelihood here.
    
    # deriv_binary_crossentropy <- t(X) %*% (y_hat - Y)
    deriv_log_likelihood <- - t(X) %*% (y_hat - Y)
    
    # The following implements RMSProp which was used in the neural network model above.
    
    # v_dtheta <- beta * v_dtheta + (1 - beta) * deriv_binary_crossentropy^2
    v_dtheta <- beta * v_dtheta + (1 - beta) * deriv_log_likelihood^2
    
    # new_theta <- new_theta - alpha * deriv_binary_crossentropy / sqrt(v_dtheta)
    theta <- theta + alpha * deriv_log_likelihood / sqrt(v_dtheta + 1e-10)
}

theta - logistic_model$coefficients

pregnant,2.514683e-02
glucose,4.719363e-03
pressure,-1.556096e-03
triceps,2.050325e-03
insulin,2.761012e-03
mass,9.211626e-04
pedigree,-6.092262e-02
age,3.491333e-05


### Conclusion: If you've used logistic regression in R before, you've used a neural network! (Though you probably shouldn't change your LinkedIn profile to Deep Learning Engineer).

### Advanced: for an in-depth illustration of what's going on when you call glm in R, see the Fisher scoring method below. For more detail, read [this paper](https://statacumen.com/teach/SC1/SC1_11_LogisticRegression.pdf).

In [27]:
manual_logistic_regression = function(X,y,threshold = 1e-10, max_iter = 100) {
  #A function to find logistic regression coeffiecients 
  #Takes three inputs:
    
    
  #A function to return p, given X and beta
  #We'll need this function in the iterative section
#   sigmoid = function(X, beta) {
#     beta = as.vector(beta)
#     return((1) / (1 + exp(-(X %*% beta))))
#   }  
 
  #### setup bit ####
 
  #initial guess for beta
  beta = rep(0, ncol(X))
 
  #initial value bigger than threshold so that we can enter our while loop 
  diff = 10000 
 
  #counter to ensure we're not stuck in an infinite loop
  iter_count = 0
  
  #### iterative bit ####
  while(diff > threshold ) #tests for convergence
  {
    # Calculate probability vector using sigmoid_fn defined above
    p = as.vector(sigmoid_fn(X %*% beta))
    
    #calculate matrix of weights W
    W =  diag(p*(1-p)) 
    
    #calculate the change in beta
    beta_change = solve(t(X)%*%W%*%X) %*% t(X)%*%(y - p)
    
    #update beta
    beta = beta + beta_change
    
    #calculate how much we changed beta by in this iteration 
    #if this is less than threshold, we'll break the while loop 
    diff = sum(beta_change^2)
    
    #see if we've hit the maximum number of iterations
    iter_count = iter_count + 1

  }

  print(iter_count)
  return(beta)
}

In [28]:
iter_count

ERROR: Error in eval(expr, envir, enclos): object 'iter_count' not found


In [29]:
manual_logistic_regression(X,Y) - logistic_model$coefficients

[1] 4


pregnant,-2.302855e-11
glucose,-3.585189e-12
pressure,1.035030e-11
triceps,9.101347e-13
insulin,1.779653e-13
mass,-7.169608e-12
pedigree,-4.444600e-11
age,1.446003e-12


In [ ]:
fisher_theta <- rep(0, k)  # Initialize theta (weights, coefficient, beta, etc.)
fisher_theta_update <- rep(1, k)  # Initialize beta_update
epsilon <- 1e-10

### Fit logistic regression using Fisher scoring ###

# While the difference between theta and theta update is greater than some threshold epsilon
while (sum(abs(fisher_theta - fisher_theta_update)) > epsilon) {
    
    # Assign theta to theta_update
    fisher_theta_update <- fisher_theta

    # Calculate mu and eta
    eta <- X %*% fisher_theta
    mu <- 1 / (1 + exp(-eta))

    # Calculate the derivatives
    #dmu_deta <- exp(-eta) * (1 / (( 1 + exp(-eta)) ^ 2))
    dmu_deta <- mu * (1 - mu)
    deta_dmu <- 1 / mu + 1 / (1 - mu)

    # Calculate the variance function. Recall that the variance of the Bernoulli distribution is p * (1 - p)
    # Since in the logistic regression setting p = 1 / (1 + (-X %*% theta)), the variance vector is defined as:
    V <- mu * (1 - mu) 
    
    # Calculate the weight matrix
    W <- diag((dmu_deta ^ 2 / V)[,]) 
    Z <- eta + (Y - mu) * deta_dmu  # Calculate Z

    # theta = (X^T %*% W %*% X)^(-1) %*% (X^T %*% W %*% Z)
    fisher_theta <- solve(t(X) %*% W %*% X) %*% (t(X) %*% W %*% Z)
    #fisher_theta <- solve(t(X) %*% W %*% X) %*% (t(X) %*% (Y - mu))
}

fisher_theta - logistic_model$coefficients

In [ ]:
eta + (Y - mu) * deta_dmu

In [ ]:
fisher_theta <- k_zeros(shape = c(0, k))  # Initialize theta (weights, coefficient, beta, etc.)
fisher_theta_update <- k_ones(shape = c(0, k))  # Initialize beta_update
epsilon <- k_eps()

while (sum(abs(fisher_theta - fisher_theta_update)) > epsilon) {
    
    # Assign theta to theta_update
    fisher_theta_update <- fisher_theta

    # Calculate mu and eta
    #eta <- X %*% fisher_theta
    eta <- k_dot(X, fisher_theta)
    
    mu <- k_sigmoid(eta)
    # mu <- 1 / (1 + exp(-eta))

    # Calculate the derivatives
    #dmu_deta <- exp(-eta) * (1 / (( 1 + exp(-eta)) ^ 2))
    dmu_deta <- 
    deta_dmu <- 1 / mu + 1 / (1 - mu)

    # Calculate the variance function. Recall that the variance of the Bernoulli distribution is p * (1 - p)
    # Since in the logistic regression setting p = 1 / (1 + (-X %*% theta)), the variance vector is defined as:
    V <- mu * (1 - mu) 
    
    # Calculate the weight matrix
    W <- diag((dmu_deta ^ 2 / V)[,]) 
    Z <- eta + (Y - mu) * deta_dmu  # Calculate Z

    # theta = (X^T %*% W %*% X)^(-1) %*% (X^T %*% W %*% Z)
    fisher_theta <- solve(t(X) %*% W %*% X) %*% (t(X) %*% W %*% Z)
}

fisher_theta - logistic_model$coefficients

In [ ]:
mu1 <- k_sigmoid(eta)

### Full disclosure: all of the hyperparameters were carefully chosen to produce the same weights in the NN model as the logistic regression model. It is by no means guaranteed to get similar weights for the 2 types of models across different hyperparameters or even across datasets.

### If it seems like we're far away from neural networks, consider what we've done:
    1. We defined a neural network with no hidden layer.
    2. We are using a form of backpropagation
    3. Our loss function is just binary crossentropy with a sign reversal.
    4. We use a particular form of optimization, rather than gradient descent.